In [2]:
!pip install ipython-autotime

%load_ext autotime

time: 194 µs (started: 2021-05-17 18:42:51 +00:00)


In [3]:
!pip install pyspark===3.1.1

     |████████████████████████████████| 212.3MB 61kB/s 
     |████████████████████████████████| 204kB 18.5MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=38c425f2017bd79ecfa0e0d1bdfe1068579aff5cf01671794b3bae4e56d910ad
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark
time: 39.4 s (started: 2021-05-17 18:42:51 +00:00)


In [4]:
!curl -O https://mars.ru77.ru/data/title.basics.tsv.gz
!curl -O https://mars.ru77.ru/data/title.crew.tsv.gz
!curl -O https://mars.ru77.ru/data/title.episode.tsv.gz
!curl -O https://mars.ru77.ru/data/title.principals.tsv.gz
!curl -O https://mars.ru77.ru/data/title.ratings.tsv.gz
!curl -O https://mars.ru77.ru/data/title.akas.tsv.gz
!curl -O https://mars.ru77.ru/data/name.basics.tsv.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  132M  100  132M    0     0  13.3M      0  0:00:09  0:00:09 --:--:-- 15.9M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 51.7M  100 51.7M    0     0  10.6M      0  0:00:04  0:00:04 --:--:-- 12.9M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 29.6M  100 29.6M    0     0  8887k      0  0:00:03  0:00:03 --:--:-- 8887k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  342M  100  342M    0     0  15.1M      0  0:00:22  0:00:22 --:--:-- 16.0M
  % Total    % Received % Xferd  Average Speed   Tim

In [5]:
from pyspark.sql import SparkSession, SQLContext
from pyspark import SparkConf, SparkContext

time: 128 ms (started: 2021-05-17 18:44:43 +00:00)


In [6]:
spark = SparkSession.builder.master("local[2]").config("spark.driver.memory", "8g").appName("aig").enableHiveSupport().getOrCreate()
sql = spark.sql

time: 8.42 s (started: 2021-05-17 18:44:43 +00:00)


In [7]:
title_basics_csv = spark.read.csv("title.basics.tsv.gz", sep='\\t', nullValue='\\N', header=True, quote="", escape="")
title_basics_csv.createOrReplaceTempView("title_basics_csv")
 
title_principals_csv = spark.read.csv("title.principals.tsv.gz", sep='\\t', nullValue='\\N', header=True, quote="", escape="")
title_principals_csv.createOrReplaceTempView("title_principals_csv")
 
title_crew_csv = spark.read.csv("title.crew.tsv.gz", sep='\\t', nullValue='\\N', header=True, quote="", escape="")
title_crew_csv.createOrReplaceTempView("title_crew_csv")
 
title_episode_csv = spark.read.csv("title.episode.tsv.gz", sep='\\t', nullValue='\\N', header=True, quote="", escape="")
title_episode_csv.createOrReplaceTempView("title_episode_csv")
 
title_ratings_csv = spark.read.csv("title.ratings.tsv.gz", sep='\\t', nullValue='\\N', header=True, quote="", escape="")
title_ratings_csv.createOrReplaceTempView("title_ratings_csv")
 
title_akas_csv = spark.read.csv("title.akas.tsv.gz", sep='\\t', nullValue='\\N', header=True, quote="", escape="")
title_akas_csv.createOrReplaceTempView("title_akas_csv")
 
name_basics_csv = spark.read.csv("name.basics.tsv.gz", sep='\\t', nullValue='\\N', header=True, quote="", escape="")
name_basics_csv.createOrReplaceTempView("name_basics_csv")

time: 9.97 s (started: 2021-05-17 18:44:52 +00:00)


In [8]:
sql('SHOW TABLES').show()

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
|        |     name_basics_csv|       true|
|        |      title_akas_csv|       true|
|        |    title_basics_csv|       true|
|        |      title_crew_csv|       true|
|        |   title_episode_csv|       true|
|        |title_principals_csv|       true|
|        |   title_ratings_csv|       true|
+--------+--------------------+-----------+

time: 12.4 s (started: 2021-05-17 18:45:02 +00:00)


# 1. Жанровые фильмы

In [9]:
sql("CREATE TABLE title_basics STORED AS PARQUET AS SELECT * FROM title_basics_csv").show()

++
||
++
++

time: 46.6 s (started: 2021-05-17 18:45:14 +00:00)


In [10]:
sql("CREATE TABLE title_basics_genres STORED AS PARQUET AS SELECT a.tconst, a.primaryTitle, EXPLODE(split(a.genres, '[,]', -1)) AS genre FROM title_basics_csv a").show()

++
||
++
++

time: 1min 5s (started: 2021-05-17 18:46:01 +00:00)


In [11]:
sql('SELECT COUNT(*) genre_count FROM title_basics_genres WHERE genre = "Comedy"').show()

+-----------+
|genre_count|
+-----------+
|    1598064|
+-----------+

time: 3.09 s (started: 2021-05-17 18:47:07 +00:00)


In [12]:
sql('SELECT COUNT(*) genre_count FROM title_basics WHERE genres LIKE "%Comedy%"').show()

+-----------+
|genre_count|
+-----------+
|    1598064|
+-----------+

time: 1.51 s (started: 2021-05-17 18:47:10 +00:00)


## 2. Хорошие жанровые фильмы

In [13]:
title_ratings_csv.show()

+---------+-------------+--------+
|   tconst|averageRating|numVotes|
+---------+-------------+--------+
|tt0000001|          5.7|    1697|
|tt0000002|          6.0|     211|
|tt0000003|          6.5|    1451|
|tt0000004|          6.1|     123|
|tt0000005|          6.1|    2248|
|tt0000006|          5.2|     125|
|tt0000007|          5.4|     687|
|tt0000008|          5.4|    1874|
|tt0000009|          6.0|     155|
|tt0000010|          6.9|    6296|
|tt0000011|          5.2|     280|
|tt0000012|          7.4|   10818|
|tt0000013|          5.7|    1623|
|tt0000014|          7.1|    4815|
|tt0000015|          6.1|     876|
|tt0000016|          5.9|    1249|
|tt0000017|          4.6|     254|
|tt0000018|          5.3|     502|
|tt0000019|          5.3|      19|
|tt0000020|          5.0|     281|
+---------+-------------+--------+
only showing top 20 rows

time: 227 ms (started: 2021-05-17 18:47:11 +00:00)


In [14]:
title_basics_csv.show()

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|   null|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|   null|             5|     Animation,Short|
|tt0000003|    short|      Pauvre Pierrot|      Pauvre Pierrot|      0|     1892|   null|             4|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|   null|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|   null|             1|        Comedy

In [15]:
sql("CREATE TABLE IF NOT EXISTS genres_ratings STORED AS PARQUET AS SELECT a.tconst, a.primaryTitle, b.averageRating, b.numVotes, EXPLODE(split(a.genres, '[,]', -1)) AS genre \
                  FROM title_basics_csv a LEFT JOIN title_ratings_csv b on a.tconst=b.tconst").show();

++
||
++
++

time: 1min 17s (started: 2021-05-17 18:47:12 +00:00)


In [16]:
sql('SELECT genre, COUNT(tconst) as count, MAX(numVotes) as max_votes FROM genres_ratings WHERE averageRating >= 4 AND genre = "Comedy" GROUP BY genre').show()

+------+------+---------+
| genre| count|max_votes|
+------+------+---------+
|Comedy|346708|     9996|
+------+------+---------+

time: 7.66 s (started: 2021-05-17 18:48:30 +00:00)


## 3. И швец, и жнец

In [17]:
title_crew_csv.show()

+---------+-------------------+---------+
|   tconst|          directors|  writers|
+---------+-------------------+---------+
|tt0000001|          nm0005690|     null|
|tt0000002|          nm0721526|     null|
|tt0000003|          nm0721526|     null|
|tt0000004|          nm0721526|     null|
|tt0000005|          nm0005690|     null|
|tt0000006|          nm0005690|     null|
|tt0000007|nm0005690,nm0374658|     null|
|tt0000008|          nm0005690|     null|
|tt0000009|          nm0085156|nm0085156|
|tt0000010|          nm0525910|     null|
|tt0000011|          nm0804434|     null|
|tt0000012|nm0525908,nm0525910|     null|
|tt0000013|          nm0525910|     null|
|tt0000014|nm0349785,nm0525910|     null|
|tt0000015|          nm0721526|     null|
|tt0000016|          nm0525910|     null|
|tt0000017|nm0804434,nm1587194|     null|
|tt0000018|          nm0804434|     null|
|tt0000019|          nm0932055|     null|
|tt0000020|          nm0010291|     null|
+---------+-------------------+---

In [18]:
name_basics_csv.show()

+---------+-------------------+---------+---------+--------------------+--------------------+
|   nconst|        primaryName|birthYear|deathYear|   primaryProfession|      knownForTitles|
+---------+-------------------+---------+---------+--------------------+--------------------+
|nm0000001|       Fred Astaire|     1899|     1987|soundtrack,actor,...|tt0031983,tt00723...|
|nm0000002|      Lauren Bacall|     1924|     2014|  actress,soundtrack|tt0117057,tt00718...|
|nm0000003|    Brigitte Bardot|     1934|     null|actress,soundtrac...|tt0057345,tt00544...|
|nm0000004|       John Belushi|     1949|     1982|actor,soundtrack,...|tt0072562,tt00804...|
|nm0000005|     Ingmar Bergman|     1918|     2007|writer,director,a...|tt0050976,tt00509...|
|nm0000006|     Ingrid Bergman|     1915|     1982|actress,soundtrac...|tt0038109,tt00777...|
|nm0000007|    Humphrey Bogart|     1899|     1957|actor,soundtrack,...|tt0043265,tt00425...|
|nm0000008|      Marlon Brando|     1924|     2004|actor,sou

In [19]:
title_akas_csv.show()

+---------+--------+--------------------+------+--------+-----------+--------------------+---------------+
|  titleId|ordering|               title|region|language|      types|          attributes|isOriginalTitle|
+---------+--------+--------------------+------+--------+-----------+--------------------+---------------+
|tt0000001|       1|          Карменсіта|    UA|    null|imdbDisplay|                null|              0|
|tt0000001|       2|          Carmencita|    DE|    null|       null|       literal title|              0|
|tt0000001|       3|Carmencita - span...|    HU|    null|imdbDisplay|                null|              0|
|tt0000001|       4|          Καρμενσίτα|    GR|    null|imdbDisplay|                null|              0|
|tt0000001|       5|          Карменсита|    RU|    null|imdbDisplay|                null|              0|
|tt0000001|       6|          Carmencita|    US|    null|       null|                null|              0|
|tt0000001|       7|          Carmenc

In [20]:
title_basics_csv.show()

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|   null|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|   null|             5|     Animation,Short|
|tt0000003|    short|      Pauvre Pierrot|      Pauvre Pierrot|      0|     1892|   null|             4|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|   null|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|   null|             1|        Comedy

In [21]:
# фильмы, с одним актером-режиссером
sql("CREATE TABLE IF NOT EXISTS writ_dir_movies STORED AS PARQUET AS SELECT DISTINCT tconst, director as director_writer FROM title_crew_csv d LATERAL VIEW EXPLODE(split(d.directors, '[,]', -1)) AS director \
                                 LATERAL VIEW EXPLODE(split(d.writers, '[,]', -1)) AS writer WHERE d.directors = d.writers").show()

++
||
++
++

time: 32.8 s (started: 2021-05-17 18:48:39 +00:00)


In [22]:
sql('SELECT COUNT(*) FROM writ_dir_movies').show()

+--------+
|count(1)|
+--------+
|  791991|
+--------+

time: 1.14 s (started: 2021-05-17 18:49:12 +00:00)


In [23]:
sql("CREATE TABLE IF NOT EXISTS movies_with_same_writ_dir AS SELECT EXPLODE(split(c.genres, '[,]', -1)) as genre, b.primaryName as director_writer, c.primaryTitle, d.title as rus_title FROM writ_dir_movies a LEFT JOIN name_basics_csv b on a.director_writer = b.nconst\
             LEFT JOIN title_basics_csv c ON c.tconst = a.tconst \
             LEFT JOIN (SELECT titleId, title FROM title_akas_csv WHERE region = 'RU') as d ON d.titleId = a.tconst").show()

++
||
++
++

time: 2min 27s (started: 2021-05-17 18:49:13 +00:00)


In [24]:
sql(""" 
        SELECT * 
        FROM movies_with_same_writ_dir
        WHERE genre = 'Comedy'
  """).show() 

+------+--------------------+--------------------+--------------------+
| genre|     director_writer|        primaryTitle|           rus_title|
+------+--------------------+--------------------+--------------------+
|Comedy|       D.W. Griffith|  His Wife's Visitor|                null|
|Comedy|Gilbert M. 'Bronc...|Alkali Ike Plays ...|                null|
|Comedy|     Charles Chaplin|  His New Profession| Его новая профессия|
|Comedy|          Max Linder|          Be My Wife|     Будь моей женой|
|Comedy|          Bud Fisher|        The Stampede|                null|
|Comedy|     Charles Chaplin|         The Pilgrim|                null|
|Comedy|        Walter Lantz|           Bull-Oney|                null|
|Comedy|          Tom Palmer|           Bull-Oney|                null|
|Comedy|     Carson Davidson|Help! My Snowman'...|                null|
|Comedy|         Andy Warhol| The Nude Restaurant|                null|
|Comedy|      Ephraim Kishon|         The Big Dig|              

## 4. Лучший в своем жанре

In [25]:
sql("CREATE TABLE IF NOT EXISTS genres_ratings_ STORED AS PARQUET AS SELECT a.tconst, a.primaryTitle, CAST(b.averageRating  AS DECIMAL(3, 1)), CAST(b.numVotes AS INT), EXPLODE(split(a.genres, '[,]', -1)) AS genre \
                  FROM title_basics_csv a LEFT JOIN title_ratings_csv b on a.tconst=b.tconst").show();

++
||
++
++

time: 1min 19s (started: 2021-05-17 18:51:42 +00:00)


In [26]:
sql('CREATE TABLE IF NOT EXISTS grouped_genres_ STORED AS PARQUET AS SELECT genre, COUNT(genre) as movies_count, SUM(averageRating*numVotes)/SUM(numVotes) avgRating FROM genres_ratings_ GROUP BY genre').show();

++
||
++
++

time: 11.3 s (started: 2021-05-17 18:53:01 +00:00)


In [27]:
sql('SELECT * FROM grouped_genres_').show(50)

+-----------+------------+-----------------+
|      genre|movies_count|        avgRating|
+-----------+------------+-----------------+
|Documentary|      730992|7.506087425332168|
| Reality-TV|      405142|6.737206815311104|
|  Animation|      385975|7.515219940048797|
|  Adventure|      301222|7.242417089821913|
|  Talk-Show|      846144|7.233471574780194|
|  Film-Noir|         781|7.626964374163787|
|  Biography|       84118|7.570546121882929|
|   Thriller|      131180|7.055362869682239|
|    Musical|       58639|7.011799637513089|
|    Western|       27716|7.536071663795618|
|    Mystery|      157535|7.290509630664947|
|    Romance|      698980|7.020704748491512|
|    History|      108148|7.609742222067169|
|     Family|      545824|6.947679026517946|
|     Sci-Fi|       95733|7.108837066225370|
|     Comedy|     1598064|6.965853243553601|
|      Music|      378161|7.129884519323754|
|      Adult|      231025|6.203866186946321|
|      Crime|      339866|7.421558660575817|
|      Dra

In [28]:
sql("""SELECT a.genre, b.movies_count, b.avgRating, a.primaryTitle as best_movie, a.averageRating as best_movie_rating
       FROM genres_ratings a 
       LEFT JOIN grouped_genres_ b on b.genre = a.genre
       WHERE a.genre = 'Comedy'
       ORDER BY averageRating DESC, numVotes DESC, tconst ASC
       LIMIT 1
        """).show()
        

+------+------------+-----------------+--------------------+-----------------+
| genre|movies_count|        avgRating|          best_movie|best_movie_rating|
+------+------------+-----------------+--------------------+-----------------+
|Comedy|     1598064|6.965853243553601|Episode dated 14 ...|              9.9|
+------+------------+-----------------+--------------------+-----------------+

time: 7.5 s (started: 2021-05-17 18:53:13 +00:00)


In [33]:
sql("""CREATE TABLE IF NOT EXISTS final_for_4_ STORED AS PARQUET AS SELECT a.tconst, a.genre, a.numVotes, b.movies_count, b.avgRating, a.primaryTitle as best_movie, a.averageRating as best_movie_rating
       FROM genres_ratings a 
       LEFT JOIN grouped_genres_ b on b.genre = a.genre
        """).show()

++
||
++
++

time: 34.3 s (started: 2021-05-17 18:58:06 +00:00)


In [38]:
sql("""SELECT genre, movies_count, avgRating, best_movie, best_movie_rating
       FROM  final_for_4_
       WHERE genre = 'Comedy'
       ORDER BY best_movie_rating DESC, numVotes DESC, tconst ASC
       LIMIT 1""").show()

+------+------------+-----------------+--------------------+-----------------+
| genre|movies_count|        avgRating|          best_movie|best_movie_rating|
+------+------------+-----------------+--------------------+-----------------+
|Comedy|     1598064|6.965853243553601|Episode dated 14 ...|              9.9|
+------+------------+-----------------+--------------------+-----------------+

time: 6 s (started: 2021-05-17 19:01:23 +00:00)


## 5. Любимчики режиссера

In [84]:
sql("""CREATE TABLE IF NOT EXISTS movie_with_dirs STORED AS PARQUET
       SELECT a.*, c.averageRating, c.numVotes, explode(split(b.directors, '[,]', - 1)) as dir
       FROM title_basics_csv a 
       LEFT JOIN title_crew_csv b on b.tconst=a.tconst
       LEFT JOIN title_ratings_csv c on a.tconst=c.tconst
       """).show()

++
||
++
++

time: 1min 50s (started: 2021-05-17 19:49:16 +00:00)


In [92]:
sql("""CREATE TABLE IF NOT EXISTS dir_data STORED AS PARQUET
       SELECT dir, COUNT(tconst) as dir_films_count, SUM(averageRating*numVotes)/SUM(numVotes) as dir_avgRating, MAX(numVotes) as dir_max_votes
       FROM movie_with_dirs
       GROUP BY dir""").show()

++
||
++
++

time: 24.5 s (started: 2021-05-17 20:01:27 +00:00)


In [108]:
sql("""CREATE TABLE IF NOT EXISTS act_dir_data STORED AS PARQUET
       SELECT a.dir, b.nconst, COUNT(a.tconst) as paired_films_count, sum(a.averageRating * a.numVotes) / sum(a.numVotes) as act_dir_avg_rating, MAX(a.numVotes) as act_dir_max_votes
       FROM movie_with_dirs a 
       LEFT JOIN (SELECT * FROM title_principals_csv WHERE Category IN ('self','actor','actress')) b on a.tconst=b.tconst
       GROUP BY a.dir, b.nconst
       """).show()

++
||
++
++

time: 3min 42s (started: 2021-05-17 20:52:22 +00:00)


In [86]:
sql("""CREATE TABLE IF NOT EXISTS actor_data STORED AS PARQUET
SELECT a.nconst, count(a.tconst) AS actor_films_count, sum(b.averageRating * b.numVotes) / sum(b.numVotes) AS avgRating_act, max(b.numVotes) AS actors_maxVotes
FROM title_principals_csv AS a
INNER JOIN (SELECT c.tconst, c.primaryTitle , d.averageRating, cast(d.numVotes AS BIGINT) AS numVotes
            FROM title_basics_csv AS c
            LEFT JOIN title_ratings_csv AS d
	          ON d.tconst = c.tconst) AS b 
ON a.tconst = b.tconst
WHERE category IN ('actor','actress','self')
GROUP BY nconst;""").show()

++
||
++
++

time: 2min 43s (started: 2021-05-17 19:51:36 +00:00)


In [112]:
sql("SELECT * FROM actor_data;").show()
sql("SELECT * FROM dir_data;").show()
sql("SELECT * FROM act_dir_data;").show()

+---------+-----------------+------------------+---------------+
|   nconst|actor_films_count|     avgRating_act|actors_maxVotes|
+---------+-----------------+------------------+---------------+
|nm0909825|               37|6.5307692307692315|            111|
|nm0606887|               50| 6.123287671232877|             29|
|nm0865222|               23| 6.039130434782607|             13|
|nm0872456|               86|  7.27026863987079|          44401|
|nm0310989|               65| 6.642885401556113|           8155|
|nm0490404|               26| 6.672641509433962|            159|
|nm0700803|              124| 5.378081082368634|           8977|
|nm0815888|                2|6.5321428571428575|             69|
|nm0093030|               64| 7.270828015914721|           9849|
|nm0906987|               49| 6.900466537966539|           1566|
|nm0460974|               52| 6.572033639143731|          18837|
|nm0098734|               33|  6.22206678700361|          11438|
|nm0282760|              

In [124]:
sql("""CREATE TABLE IF NOT EXISTS lubimki STORED AS PARQUET AS 
       SELECT ad.*, d.dir_films_count, d.dir_avgRating, d.dir_max_votes, a.actor_films_count, a.avgRating_act, a.actors_maxVotes, n1.primaryName as actor_name, n2.primaryName as director_name
       FROM act_dir_data ad
       LEFT JOIN dir_data d on ad.dir = d.dir
       LEFT JOIN actor_data a on a.nconst = ad.nconst
       LEFT JOIN name_basics_csv n1 on n1.nconst = ad.nconst
       LEFT JOIN name_basics_csv n2 on n2.nconst = ad.dir
       """).show()

++
||
++
++

time: 2min 50s (started: 2021-05-17 21:15:48 +00:00)


In [129]:
sql("""SELECT director_name,
              dir_films_count director_films_count,
              dir_avgRating director_avgRating,
              dir_max_votes director_max_votes,
              actor_name, 
              actor_films_count,
              avgRating_act actor_avgRating, 
              actors_maxVotes actor_max_votes, 
              paired_films_count, 
              act_dir_avg_rating paired_avgRating, 
              act_dir_max_votes paired_max_votes
       FROM lubimki
       WHERE dir = 'nm0000165'
              """).show(50)

+-------------+--------------------+------------------+------------------+--------------------+-----------------+------------------+---------------+------------------+------------------+----------------+
|director_name|director_films_count|director_avgRating|director_max_votes|          actor_name|actor_films_count|   actor_avgRating|actor_max_votes|paired_films_count|  paired_avgRating|paired_max_votes|
+-------------+--------------------+------------------+------------------+--------------------+-----------------+------------------+---------------+------------------+------------------+----------------+
|   Ron Howard|                  54| 7.262105048333398|               883|        Dianne Wiest|               73| 7.337900041285526|         451581|                 1|               7.0|           45814|
|   Ron Howard|                  54| 7.262105048333398|               883|         Bette Davis|              240| 7.634869865231024|         122053|                 1|               7.